In [111]:
# Importar librerias necesarias
import camelot          # para extraer tablas de archivos
import pandas as pd     # para manipulacion y analisis de datos

#### Extracción de tablas

In [2]:
path = "input/relacion-de-postulantes-preseleccionados.pdf"
tables = camelot.read_pdf(filepath=path, pages='all', flavor='stream')

#### Limpieza

In [9]:
# Crear un DataFrame vacío para almacenar todas las tablas
all_tables = pd.DataFrame()

# Iterar sobre todas las tablas y agregar cada DataFrame al DataFrame global
for table in tables:
    all_tables = pd.concat([all_tables, table.df], ignore_index=True)

In [10]:
all_tables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31627 entries, 0 to 31626
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       31627 non-null  object
 1   1       31627 non-null  object
 2   2       31627 non-null  object
 3   3       31627 non-null  object
 4   4       31627 non-null  object
 5   5       31627 non-null  object
 6   6       31627 non-null  object
 7   7       31627 non-null  object
 8   8       31627 non-null  object
 9   9       2267 non-null   object
dtypes: object(10)
memory usage: 2.4+ MB


In [42]:
# Filtrar y eliminar filas según campos no deseados
filter_tabs = all_tables[
    ~all_tables[all_tables.columns[0]].str.startswith(('N°', 'TABLA', 'Esta', 'Final', 'Su')) &
    ~all_tables[all_tables.columns[1]].str.startswith('TABLA') &                            # Filtrar valores en la segunda columna
    ~all_tables[all_tables.columns[4]].str.startswith(('ANEXO', 'RESOLUCIÓN', 'BECA')) &    # Filtrar valores en la cuarta columna
    ~all_tables[all_tables.columns[6]].isin(['CONDICIONES', 'PRIORIZABLES']) &              # Filtrar valores en la sexta columna
    ~all_tables[all_tables.columns[7]].isin(['CONDICIONES', 'PRIORIZABLES'])                # Filtrar valores en la séptima columna
]

In [43]:
filter_tabs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29154 entries, 3 to 31623
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       29154 non-null  object
 1   1       29154 non-null  object
 2   2       29154 non-null  object
 3   3       29154 non-null  object
 4   4       29154 non-null  object
 5   5       29154 non-null  object
 6   6       29154 non-null  object
 7   7       29154 non-null  object
 8   8       29154 non-null  object
 9   9       2118 non-null   object
dtypes: object(10)
memory usage: 2.4+ MB


Hay 2118 registros no nulos en la última columna. Se averigua cuáles son esos registros.

Los campos a exportar son:
1. Número
2. Modalidad de beca
3. DNI
4. Región
5. Puntaje Examen Nacional
6. Puntaje Adicional
7. Puntaje Final


In [27]:
filter_tabs[filter_tabs.iloc[:, 9].notna()] # Estos registros son los 2118 que tienen una columna adicional donde la 3ra columna es la POBLACION 

,0,1,2,3,4,5,6,7,8,9
22714,1,Beca CNA y PA,Comunidad Nativa Amazónica,61275706,ABAD AQUINO LEYDI SAMIRA,TACNA,64,7,71,PRESELECCIONADO
22715,2,Beca CNA y PA,Comunidad Nativa Amazónica,76837229,ABAD TORIBIO ALEXANDRA ANHELY,JUNIN,58,7,65,PRESELECCIONADO
22716,3,Beca CNA y PA,Comunidad Nativa Amazónica,60132472,ABANTO CHASHNAMOTE RUZEL,SAN MARTIN,48,12,60,PRESELECCIONADO
22717,4,Beca CNA y PA,Comunidad Nativa Amazónica,74493877,ABENSUR BUENDIA MIKHAL ANGELINA,UCAYALI,70,10,80,PRESELECCIONADO
22718,5,Beca CNA y PA,Comunidad Nativa Amazónica,60565492,ABILA GASPAR DADDY ALEXIS,JUNIN,68,4,72,PRESELECCIONADO
...,...,...,...,...,...,...,...,...,...,...
25067,2114,Beca CNA y PA,Población afroperuana,60247237,ZEVALLOS MISAICO LUIS GEARITSON,ICA,62,0,62,PRESELECCIONADO
25068,2115,Beca CNA y PA,Población afroperuana,62220383,ZURITA GARCIA DELIA,PIURA,42,4,46,PRESELECCIONADO
25069,2116,Beca CNA y PA,Población afroperuana,75159215,ZURITA GARCIA DUBERLY,PIURA,32,7,39,PRESELECCIONADO
25070,2117,Beca CNA y PA,Población afroperuana,60478854,ZURITA ROMAN GREYLI YOCSANY,PIURA,44,7,51,PRESELECCIONADO


In [32]:
filter_tabs[1].unique()

array(['Beca 18 ORDINARIA', '', 'Beca CNA y PA', 'Beca EIB', 'Beca FFAA',
       'Beca HUALLAGA', 'BECA PROTECCIÓN', 'BECA REPARED', 'BECA VRAEM'],
      dtype=object)

El array anterior tiene un valor de ''. Esto significa que existen registros que no tienen el nombre de tipo de beca.

In [96]:
filter_tabs[filter_tabs[1] == '']

,0,1,2,3,4,5,6,7,8,9
1978,,,,BARRIENTOS ALTAMIRANO JAHAZIEL HASHEM,,,,,,NaN
1980,,,,AQUILES,,,,,,NaN
2042,,,,BATALLANOS ARAPA SHIOMARA DEL ROSARIO,,,,,,NaN
2044,,,,GUADALUPE,,,,,,NaN
11963,,,,MAMANI COAQUERA YÚRICO YAMILETH DE LOS,,,,,,NaN
11965,,,,ANGELES,,,,,,NaN
13644,,,,MOTTOCCANCHI CHOQQUEMAMANI ROXANA,,,,,,NaN
13646,,,,GRACIELA,,,,,,NaN
15196,,,,PAZ GREGORIANO FIORELLA EMPERATRIZ,,,,,,NaN
15198,,,,KATHERINET,,,,,,NaN


Entonces, estos registros sí tienen beca solo que el postulante al tener un nombre extenso se cuenta como un registro adicional sin tomar el campo de beca y los demás. Por ejemplo, algunos postulantes se buscan manualmente por su nombre en el pdf y se valida si ya están almacenados en filter_tabs.

In [97]:
filter_tabs[filter_tabs[2]=='60220251']


,0,1,2,3,4,5,6,7,8,9
28787,1334,BECA REPARED,60220251,,JUNIN,52,12,64,PRESELECCIONADO,NaN


In [98]:
filter_tabs[filter_tabs[2]=='60167902']

,0,1,2,3,4,5,6,7,8,9
11964,10998,Beca 18 ORDINARIA,60167902,,TACNA,64,10,74,PRESELECCIONADO,NaN


In [188]:
# Lista de todas las becas
ls_becas = {
    'Beca 18 ORDINARIA': 'df_beca18',
    'Beca CNA y PA': 'df_becaCNA',
    'Beca EIB': 'df_becaEIB',
    'Beca FFAA': 'df_becaFFAA',
    'Beca HUALLAGA': 'df_becaHUA',
    'BECA PROTECCIÓN': 'df_becaPTR',
    'BECA REPARED': 'df_becaRPD',
    'BECA VRAEM': 'df_becaVRM',
}

# Crear un diccionario vacío para almacenar los DataFrames
df_becas = {}

# Recorrer ls_becas y asignar cada DataFrame a su clave correspondiente
for beca, df_name in ls_becas.items():
    df_becas[df_name] = filter_tabs[filter_tabs[1] == beca]


In [189]:
# Cantidad de filas y columnas
def showShape(df, lista):
    print('Cantidad de filas y columnas')
    for beca, df_name in lista.items():
        print(f' - {beca}: {df[df_name].shape}')


# Valores unicos del ultimo campo
def showUniqueValues(df, lista):
    print('Valores unicos de ultimo campo')
    for beca, df_name in lista.items():
        print(f' - {beca}: {df[df_name].iloc[:,-1].unique()}')

def deleteColumns(df, lista, cols):
    # Recorrer el diccionario df_becas y eliminar la última columna de cada DataFrame excepto df_becaCNA
    for beca, df_name in lista.items():
        # Verificar si el DataFrame no es df_becaCNA
        if df_name != 'df_becaCNA':
            df[df_name] = df[df_name].drop(df[df_name].columns[cols], axis=1)

    # Ahora cada DataFrame en df_becas, excepto df_becaCNA, tendrá la última columna eliminada

def deleteColsCNA(df, lista, cols):
    for beca, df_name in lista.items():
        if df_name == 'df_becaCNA':
            df[df_name] = df[df_name].drop(df[df_name].columns[cols], axis=1)

In [190]:
showShape(df_becas, ls_becas)
print()
showUniqueValues(df_becas, ls_becas)

Cantidad de filas y columnas
 - Beca 18 ORDINARIA: (20936, 10)
 - Beca CNA y PA: (2206, 10)
 - Beca EIB: (647, 10)
 - Beca FFAA: (430, 10)
 - Beca HUALLAGA: (842, 10)
 - BECA PROTECCIÓN: (102, 10)
 - BECA REPARED: (3123, 10)
 - BECA VRAEM: (852, 10)

Valores unicos de ultimo campo
 - Beca 18 ORDINARIA: [nan]
 - Beca CNA y PA: ['PRESELECCIONADO' nan]
 - Beca EIB: [nan]
 - Beca FFAA: [nan]
 - Beca HUALLAGA: [nan]
 - BECA PROTECCIÓN: [nan]
 - BECA REPARED: [nan]
 - BECA VRAEM: [nan]


Se observa que solo en el dataset de beca CNA y PA hay registros que tienen en la ultima columna el valor de PRESELECCIONADO y el resto vacíos. Se limpia este dataset por separado. Por otro lado, se eliminará la última columna en los demás datasets.

In [191]:
deleteColumns(df_becas, ls_becas, [3,-1])   # Elimina la columna de Nombres y Apellidos y la ultima columna de nan

In [192]:
showShape(df_becas, ls_becas)
print()
showUniqueValues(df_becas, ls_becas)

Cantidad de filas y columnas
 - Beca 18 ORDINARIA: (20936, 8)
 - Beca CNA y PA: (2206, 10)
 - Beca EIB: (647, 8)
 - Beca FFAA: (430, 8)
 - Beca HUALLAGA: (842, 8)
 - BECA PROTECCIÓN: (102, 8)
 - BECA REPARED: (3123, 8)
 - BECA VRAEM: (852, 8)

Valores unicos de ultimo campo
 - Beca 18 ORDINARIA: ['PRESELECCIONADO']
 - Beca CNA y PA: ['PRESELECCIONADO' nan]
 - Beca EIB: ['PRESELECCIONADO']
 - Beca FFAA: ['PRESELECCIONADO']
 - Beca HUALLAGA: ['PRESELECCIONADO']
 - BECA PROTECCIÓN: ['PRESELECCIONADO']
 - BECA REPARED: ['PRESELECCIONADO']
 - BECA VRAEM: ['PRESELECCIONADO']


In [193]:
deleteColumns(df_becas, ls_becas, [-1]) # Elimina la ultima columna de PRESELECCIONADOS

In [194]:
showShape(df_becas, ls_becas)

Cantidad de filas y columnas
 - Beca 18 ORDINARIA: (20936, 7)
 - Beca CNA y PA: (2206, 10)
 - Beca EIB: (647, 7)
 - Beca FFAA: (430, 7)
 - Beca HUALLAGA: (842, 7)
 - BECA PROTECCIÓN: (102, 7)
 - BECA REPARED: (3123, 7)
 - BECA VRAEM: (852, 7)


##### Limpiando el df CNA y PA

In [196]:
df_becas['df_becaCNA'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2206 entries, 22714 to 25071
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       2206 non-null   object
 1   1       2206 non-null   object
 2   2       2206 non-null   object
 3   3       2206 non-null   object
 4   4       2206 non-null   object
 5   5       2206 non-null   object
 6   6       2206 non-null   object
 7   7       2206 non-null   object
 8   8       2206 non-null   object
 9   9       2118 non-null   object
dtypes: object(10)
memory usage: 189.6+ KB


In [197]:
df_becaCNA_1 = df_becas['df_becaCNA'][df_becas['df_becaCNA'].iloc[:,9].notna()]
df_becaCNA_2 = df_becas['df_becaCNA'][~df_becas['df_becaCNA'].iloc[:,9].notna()]

In [198]:
df_becaCNA_1

,0,1,2,3,4,5,6,7,8,9
22714,1,Beca CNA y PA,Comunidad Nativa Amazónica,61275706,ABAD AQUINO LEYDI SAMIRA,TACNA,64,7,71,PRESELECCIONADO
22715,2,Beca CNA y PA,Comunidad Nativa Amazónica,76837229,ABAD TORIBIO ALEXANDRA ANHELY,JUNIN,58,7,65,PRESELECCIONADO
22716,3,Beca CNA y PA,Comunidad Nativa Amazónica,60132472,ABANTO CHASHNAMOTE RUZEL,SAN MARTIN,48,12,60,PRESELECCIONADO
22717,4,Beca CNA y PA,Comunidad Nativa Amazónica,74493877,ABENSUR BUENDIA MIKHAL ANGELINA,UCAYALI,70,10,80,PRESELECCIONADO
22718,5,Beca CNA y PA,Comunidad Nativa Amazónica,60565492,ABILA GASPAR DADDY ALEXIS,JUNIN,68,4,72,PRESELECCIONADO
...,...,...,...,...,...,...,...,...,...,...
25067,2114,Beca CNA y PA,Población afroperuana,60247237,ZEVALLOS MISAICO LUIS GEARITSON,ICA,62,0,62,PRESELECCIONADO
25068,2115,Beca CNA y PA,Población afroperuana,62220383,ZURITA GARCIA DELIA,PIURA,42,4,46,PRESELECCIONADO
25069,2116,Beca CNA y PA,Población afroperuana,75159215,ZURITA GARCIA DUBERLY,PIURA,32,7,39,PRESELECCIONADO
25070,2117,Beca CNA y PA,Población afroperuana,60478854,ZURITA ROMAN GREYLI YOCSANY,PIURA,44,7,51,PRESELECCIONADO


In [199]:
df_becaCNA_2

,0,1,2,3,4,5,6,7,8,9
22809,89,Beca CNA y PA,Comunidad Nativa Amazónica,60438460,APANJORAY ELESCANO YULIANA NATALY,JUNIN,60,7,67,NaN
22810,90,Beca CNA y PA,Comunidad Nativa Amazónica,61604339,APARICIO HILARIO LUCIANA,JUNIN,94,2,96,NaN
22811,91,Beca CNA y PA,Comunidad Nativa Amazónica,61036840,APAZA ZAMUDIO CALEB OTOMIEL,JUNIN,88,2,90,NaN
22812,92,Beca CNA y PA,Comunidad Nativa Amazónica,63418030,AQUINO CAMAÑA RUDY DEYVIS,PASCO,68,10,78,NaN
22813,93,Beca CNA y PA,Comunidad Nativa Amazónica,61169694,AQUITUARI PINEDO LANDER FIDEL,LORETO,68,10,78,NaN
...,...,...,...,...,...,...,...,...,...,...
23835,1007,Beca CNA y PA,Comunidad Nativa Amazónica,61948013,PASQUEL ROJAS JHILIAN JHANNY,PASCO,58,15,73,NaN
23836,1008,Beca CNA y PA,Comunidad Nativa Amazónica,60534344,PASTOR ZAMBACHE MARIA CELESTE,PASCO,70,10,80,NaN
23837,1009,Beca CNA y PA,Comunidad Nativa Amazónica,60622963,PAUCAR DE LA CRUZ MAYLI ALEJANDRINA,JUNIN,56,7,63,NaN
23838,1010,Beca CNA y PA,Comunidad Nativa Amazónica,60465241,PAUCAR OCAÑO JOSELIN NAYELY,JUNIN,60,7,67,NaN


Se observa que la unica diferencia entre estos dos datasets es que el segundo no tiene el valor de PRESELECCIONADO, el resto de campos son iguales y tiene la misma estructura. Entonces, se debe eliminar los campos 2,4,9.

In [200]:
deleteColsCNA(df_becas, ls_becas, [2,4,9])

In [202]:
showShape(df_becas, ls_becas)

Cantidad de filas y columnas
 - Beca 18 ORDINARIA: (20936, 7)
 - Beca CNA y PA: (2206, 7)
 - Beca EIB: (647, 7)
 - Beca FFAA: (430, 7)
 - Beca HUALLAGA: (842, 7)
 - BECA PROTECCIÓN: (102, 7)
 - BECA REPARED: (3123, 7)
 - BECA VRAEM: (852, 7)


#### Exportando

In [210]:
# Lista de nuevos encabezados
titles = ['N°','MODALIDAD','DNI','REGION','PJ ENP', 'PJ CP', 'PJ FIN']

# Recorrer el diccionario df_becas y asignar los nuevos encabezados a cada DataFrame
for beca, df_name in df_becas.items():
    # Asignar la lista 'titles' como los nuevos encabezados
    df_name.columns = titles

In [ ]:
# Probando...
df_becas['df_becaCNA'].head(5)

,N°,MODALIDAD,DNI,REGION,PJ ENP,PJ CP,PJ FIN
22714,1,Beca CNA y PA,61275706,TACNA,64,7,71
22715,2,Beca CNA y PA,76837229,JUNIN,58,7,65
22716,3,Beca CNA y PA,60132472,SAN MARTIN,48,12,60
22717,4,Beca CNA y PA,74493877,UCAYALI,70,10,80
22718,5,Beca CNA y PA,60565492,JUNIN,68,4,72


Se debe concatenar todos los dfs en uno. Todos tienen 7 columnas.

In [212]:
concatenated_df = pd.concat(df_becas.values(), ignore_index=True)
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29138 entries, 0 to 29137
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   N°         29138 non-null  object
 1   MODALIDAD  29138 non-null  object
 2   DNI        29138 non-null  object
 3   REGION     29138 non-null  object
 4   PJ ENP     29138 non-null  object
 5   PJ CP      29138 non-null  object
 6   PJ FIN     29138 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


Se observa que hay registros donde no se ha leído correctamente el nombre de la REGION.

In [214]:
concatenated_df[concatenated_df.REGION=='']

,N°,MODALIDAD,DNI,REGION,PJ ENP,PJ CP,PJ FIN
3448,3413,Beca 18 ORDINARIA,60720968,,72,0,72
4530,4495,Beca 18 ORDINARIA,60168545,,80,12,92
6200,6165,Beca 18 ORDINARIA,80807959,,78,0,78
8880,8845,Beca 18 ORDINARIA,76507510,,72,0,72
10959,10924,Beca 18 ORDINARIA,60918474,,102,0,102
14773,14738,Beca 18 ORDINARIA,61225074,,96,5,101
15940,15905,Beca 18 ORDINARIA,61028191,,70,7,77
21553,574,Beca CNA y PA,60540632,,62,7,69
25596,398,BECA REPARED,76414131,,58,0,58


In [215]:
# Se registra manualmente las regiones para los registros faltantes 
faltantes = {
    '60720968': 'LIMA',
    '60168545': 'AREQUIPA',
    '80807959': 'N/H',
    '76507510': 'LIMA',
    '60918474': 'LIMA',
    '61225074': 'LIMA',
    '61028191': 'HUANCAVELICA',
    '76414131': 'LIMA',
    '60540632': 'CUSCO'
}

La columna REGION se actualiza con los valores correspondientes de faltantes en función de las llaves de la columna DNI

In [216]:
# Actualizar solo las filas donde el ID coincide con las llaves del diccionario
concatenated_df.loc[concatenated_df['DNI'].isin(faltantes.keys()), 'REGION'] = concatenated_df['DNI'].map(faltantes)

In [217]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29138 entries, 0 to 29137
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   N°         29138 non-null  object
 1   MODALIDAD  29138 non-null  object
 2   DNI        29138 non-null  object
 3   REGION     29138 non-null  object
 4   PJ ENP     29138 non-null  object
 5   PJ CP      29138 non-null  object
 6   PJ FIN     29138 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [218]:
# Reemplazar las cadenas "Beca 18 ", "Beca " o "BECA " con nada (vacío)
concatenated_df['MODALIDAD'] = concatenated_df['MODALIDAD'].str.replace(r'^(Beca 18 |Beca |BECA )', '', regex=True)

In [219]:
regiones = concatenated_df['REGION'].unique()
modalidades = concatenated_df['MODALIDAD'].unique()

In [223]:
mod_df = pd.DataFrame(modalidades, columns=['MODALIDAD'])
reg_df = pd.DataFrame(regiones, columns=['REGION'])

Exportamos los archivos .csv

In [ ]:
mod_df.to_csv('output/modalidades.csv', index=False)
reg_df.to_csv('output/regiones.csv', index=False)
concatenated_df.to_csv('output/data.csv', index=False)